In [ ]:
from datetime import datetime

In [2]:
# Importy bibliotek
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

# Pliki CSV umieszczone w folderze data
uczestnicy = pd.read_csv('data/PPK_Uczestnicy.csv', sep=';')
pracodawcy = pd.read_csv('data/PPK_Pracodawcy.csv',sep=';')

# Mergowanie dwóch csv
df = pd.merge(
    uczestnicy,
    pracodawcy,
    how='left',
    left_on='EMPL_ID',
    right_on='ID'
)

# Dropowanie niepotrzebnych kolumn
df.drop(columns=['MEMBER_ID','LOGICAL_FACTOR_1','LOGICAL_FACTOR_2','ID','PKD_CODE','PPK_BANK','NUMERICAL_VALUE'],inplace=True)
# Grouping other nationalities into one category '0'
unique_nat = df['NATIONALITY'].unique()
for val in unique_nat:
    suma = ( df['NATIONALITY'].values == val ).sum()
    if suma < 500:
        df.loc[df.NATIONALITY == val, 'NATIONALITY'] = 0
# zamian danych kategorycznych na dummy variables - tworzymy nową kolumnę na każdy możliwy output kolumny, porównaj poprzednią i następną komórkę
df = pd.get_dummies(df, columns = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP'], prefix = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP'])

In [7]:
# najwcześniejsza data z "CREATED_AT"
start_date = df.iloc[:,6].min()
# nie przydała się ale zostawię, może sie przydać

In [8]:
def find_period(data):
    start, stop = data
    if type(stop) is float:
        stop = "2020-05-21"  # data otrzymania danych
    
    start_year, start_month, start_day = map(int, start.split('-'))
    stop_year, stop_month, stop_day = map(int, stop.split('-'))
    
    days = (stop_year - start_year)*365
    days += (stop_month - start_month)*30 if stop_month > start_month else (start_month - stop_month)*30
    days += stop_day - start_day if stop_day > start_day else start_day - stop_day
    
    return days


df['DURATION'] = list(zip(df['SIGN_DATE'], df['RESIGN_DATE']))

df['DURATION'] = df['DURATION'].apply(find_period)  #.astype('int32')   chciałem zmienić na inta ale nie działa 

In [11]:
df['DURATION'].describe()

count    1.793211e+06
mean     2.184507e+02
std      1.645398e+02
min      0.000000e+00
25%      6.700000e+01
50%      1.660000e+02
75%      3.710000e+02
max      7.250000e+02
Name: DURATION, dtype: float64